In [1]:
!pip install numpy pandas matplotlib scikit-learn opencv-python tensorflow keras opencv-python-headless

In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-09-27 18:42:09.753761: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-27 18:42:09.773706: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-27 18:42:09.920300: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-27 18:42:10.048248: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-27 18:42:10.157466: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

Coleta e processamento de dados

In [3]:
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, (224, 224))
    img_normalized = img_resized / 255.0
    return img_normalized

image = preprocess_image('data_set/000022.jpg')

Criando CNN (rede neural convolucional), usada para visão computacional e analise de imagens

In [10]:
from tensorflow.keras import layers, models

def create_cnn(input_shape):
    model = models.Sequential()

    # Definir a forma de entrada explicitamente com Input
    model.add(layers.Input(shape=input_shape))

    # Camada convolucional + ReLU + Pooling
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())  # Achata a saída para alimentar a camada densa
    model.add(layers.Dense(128, activation='relu'))  # Camada densa
    model.add(layers.Dense(3, activation='softmax'))

    return model

# Definindo a forma de entrada da imagem (altura, largura, canais)
input_shape = (224, 224, 1)  # 1 canal para imagens em escala de cinza
cnn_model = create_cnn(input_shape)

# Compilando o modelo
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Treinamento do modelo

In [13]:
# Gerador de dados com aumentação de imagem
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.5
)

# Diretório das imagens de treino e validação
train_generator = train_datagen.flow_from_directory(
    './data_set',
    target_size=(224, 224),
    color_mode='grayscale',
    batch_size=1, # mudar para 8 quando tiver dataset completo
    # aumenta o tamanho de processamento por vez, reduzindo peculiaridades de uma única imagem afetem a acurácia do modelo
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    './data_set',
    target_size=(224, 224),
    color_mode='grayscale',
    batch_size=1, # mudar para 8 quando tiver dataset completo
    # aumenta o tamanho de processamento por vez, reduzindo peculiaridades de uma única imagem afetem a acurácia do modelo
    class_mode='categorical',
    subset='validation'
)

# Treinamento do modelo
history = cnn_model.fit(train_generator, validation_data=validation_generator, epochs=30)

Found 3 images belonging to 3 classes.
Found 3 images belonging to 3 classes.
Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 240ms/step - accuracy: 0.2917 - loss: 1.0642 - val_accuracy: 0.6667 - val_loss: 0.7301
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.7083 - loss: 0.8743 - val_accuracy: 0.0000e+00 - val_loss: 1.2363
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - accuracy: 0.7083 - loss: 0.8543 - val_accuracy: 0.3333 - val_loss: 1.0409
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - accuracy: 0.8333 - loss: 0.6512 - val_accuracy: 0.6667 - val_loss: 0.7351
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - accuracy: 0.4583 - loss: 1.1511 - val_accuracy: 0.0000e+00 - val_loss: 1.3703
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - accuracy: 0.1667 - loss: 1.2869 - val_accuracy: 0.0000e+00 - val_loss: 1.7750
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - accuracy: 0.5417 - loss: 1.0685 - val_accuracy: 0.0000e+00 - val_loss: 1.2966
Epoch 8/30
3/3 ━━━━━━━━━

Testes de validação

In [16]:
# Avaliando o modelo no conjunto de validação
val_loss, val_acc = cnn_model.evaluate(validation_generator)
print(f"Validação - Loss: {val_loss}, Acurácia: {val_acc}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.2917 - loss: 1.8705   
Validação - Loss: 1.5646963119506836, Acurácia: 0.3333333432674408


Função de predição

In [21]:
def predict_image(image_path, model):
    img = preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)  # Adiciona uma dimensão para batch
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)  # Retorna o índice da classe com maior probabilidade
    return predicted_class, prediction

predicted_class, result = predict_image('./test-data/001.jpg', cnn_model)
print(f'Predição: {result}')
print(f'Classe prevista: {predicted_class}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predição: [[0.11641166 0.10216209 0.7814262 ]]
Classe prevista: 2
